# 02-Training-PyTorch-GNN

## 1. Loading dataset

In [1]:
from torch_geometric.datasets import TUDataset

# download dataset
dataset = TUDataset(root="../datasets/", name="IMDB-BINARY")

## 2. Traing Node2Vec model

In [11]:
import numpy as np
import torch
from torch_geometric.nn import Node2Vec

device = 'cpu'
data = dataset
data.y = data.y

# Train Node2Vec model
emb_model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=30,
                     context_size=30, walks_per_node=20,
                     num_negative_samples=1, p=0.5, q=0.5).to(device)

loader = emb_model.loader(batch_size=128, shuffle=True, num_workers=0)
optimizer = torch.optim.Adam(emb_model.parameters(), lr=0.01)

In [12]:
def train_test_split(test_ratio=0.1):
    y = data.data.y
    test_size = int(len(y) * test_ratio)
    classes = np.unique(y)

    train_idx = []
    test_idx = []

    for cls in classes:
        idx = (y == cls).nonzero().view(-1)
        idx = idx[torch.randperm(idx.size(0))]
        train_idx.extend(idx[:-test_size])
        test_idx.extend(idx[-test_size:])

    train_idx = torch.tensor(train_idx).to(device)
    test_idx = torch.tensor(test_idx).to(device)

    return train_idx, test_idx


train_idx, test_idx = train_test_split(test_ratio=0.1)

In [13]:
def train(epoch, model):
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in tqdm(loader, desc=f'Epoch {epoch:03d}'):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


for epoch in range(1, 21):
    loss = train(epoch, emb_model)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch 001: 100%|██████████| 155/155 [00:05<00:00, 29.76it/s]


Epoch: 001, Loss: 8.1273


Epoch 002: 100%|██████████| 155/155 [00:05<00:00, 29.83it/s]


Epoch: 002, Loss: 5.1322


Epoch 003: 100%|██████████| 155/155 [00:05<00:00, 29.89it/s]


...

Epoch: 018, Loss: 0.7997


Epoch 019: 100%|██████████| 155/155 [00:05<00:00, 29.83it/s]


Epoch: 019, Loss: 0.7853


Epoch 020: 100%|██████████| 155/155 [00:05<00:00, 29.83it/s]

Epoch: 020, Loss: 0.7742


In [14]:
import pickle

with open('../models/node2vec.pkl', 'wb') as f:
    pickle.dump(emb_model.state_dict(), f)

## 3. Assigning node embeddings to nodes in dataset

In [15]:
import pickle

with open('../models/node2vec.pkl', 'rb') as f:
    emb_model.load_state_dict(state_dict=pickle.load(f))

In [16]:
node_embeddings = emb_model()

In [17]:
from torch_geometric.io import read_tu_data


class FilmsDataset(TUDataset):
    def __init__(self, root: str, name: str):
        super().__init__(root, name)

    def process(self):
        self.data, self.slices, sizes = read_tu_data(self.raw_dir, self.name)

        data_list = [self.get(idx) for idx in range(len(self))]
        num_nodes = 0
        for data in data_list:
            data.x = node_embeddings[num_nodes: num_nodes + data.num_nodes]
            num_nodes += data.num_nodes

        self.data, self.slices = self.collate(data_list)
        self._data_list = None

        torch.save((self._data.to_dict(), self.slices, sizes),
                   self.processed_paths[0])

In [18]:
dataset = FilmsDataset(root="../datasets/", name="IMDB-BINARY")

## 4. GCN Class

In [19]:
import torch
from torch import nn
from torch.nn import functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

In [20]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList([GCNConv(input_dim, hidden_dim)] + \
                                         [GCNConv(hidden_dim, hidden_dim) for i in range(num_layers - 2)] + \
                                         [GCNConv(hidden_dim, output_dim)])

        self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for i in range(num_layers - 1)])

        self.softmax = torch.nn.LogSoftmax(dim=-1)

        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i in range(len(self.convs) - 1):
            x = self.convs[i](x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.convs[-1](x, adj_t)
        if self.return_embeds:
            out = x
        else:
            out = self.softmax(x)

        return out

In [21]:
class GCN_Graph(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        self.gnn_node = GCN(input_dim, hidden_dim,
                            hidden_dim, num_layers, dropout, return_embeds=True)

        # Pooling layer
        self.pool = global_mean_pool

        # Output layer
        self.linear = torch.nn.Linear(hidden_dim, output_dim)

    def reset_parameters(self):
        self.gnn_node.reset_parameters()
        self.linear.reset_parameters()

    def forward(self, batched_data):
        # Extract important attributes of our mini-batch
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch

        embed = self.gnn_node(x, edge_index)

        out = self.pool(embed, batch)

        out = self.linear(out)

        return out

## 5. Preparing data

In [22]:
train_dataset, test_dataset = dataset[train_idx], dataset[test_idx]

In [23]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

## 6. Training

In [24]:
model = GCN_Graph(input_dim=256, hidden_dim=128, output_dim=dataset.num_classes,
                  num_layers=2, dropout=0.2).to(device)

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [35]:
from tqdm import tqdm


def train(epoch, model, optimizer, criterion, train_loader):
    model.train()

    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()

        out = model(batch.to(device))

        loss = criterion(out, batch.y)
        loss.backward()

        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

In [36]:
def test(model, test_loader):
    model.eval()

    correct = 0
    for batch in test_loader:
        with torch.no_grad():
            out = model(batch.to(device))

        pred = out.argmax(dim=1)
        correct += int((pred == batch.y).sum())

    return correct / len(test_loader.dataset)

In [37]:
epochs = 50
model.reset_parameters()
for epoch in range(epochs):
    loss = train(epoch, model, optimizer, criterion, train_loader)
    acc = test(model, test_loader)
    print(f"Epoch: {epoch + 1:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}")

Epoch 1: 100%|██████████| 7/7 [00:00<00:00, 41.05it/s]


Epoch: 001, Loss: 0.7206, Acc: 0.5250


Epoch 2: 100%|██████████| 7/7 [00:00<00:00, 40.45it/s]


Epoch: 002, Loss: 0.6608, Acc: 0.5750


Epoch 3: 100%|██████████| 7/7 [00:00<00:00, 39.55it/s]


...

Epoch: 048, Loss: 0.0020, Acc: 0.5600


Epoch 49: 100%|██████████| 7/7 [00:00<00:00, 40.33it/s]


Epoch: 049, Loss: 0.0020, Acc: 0.5550


Epoch 50: 100%|██████████| 7/7 [00:00<00:00, 41.25it/s]


Epoch: 050, Loss: 0.0013, Acc: 0.5450


# Credits

Notebook was made by GCN-tutorial team for the Innopolis 'Data and Knowledge representation' course.

> [Polina Zelenskaya](github.com/cutefluffyfox) \
> [Said Kamalov](github.com/SaidKamalov) \
> [Lev Rekhlov](github.com/plov-cyber)